# Week 36

1. Explore the dataset from https://huggingface.co/datasets/coastalcph/tydi_xor_rc. Familiarize yourself with the dataset card, download the
dataset and explore its columns. Summarize basic data statistics for training and validation data in each of the languages Finnish (fi), Japanese
(ja) and Russian (ru).

2. For each of the languages Finnish, Japanese and Russian, report the 5
most common words in the questions from the training set. What kind of
words are they?

3. Implement a rule-based classifier that predicts whether a question is answerable or impossible, only using the document (context) and question.
You may use machine translation as a component. Use the answerable
field to evaluate it on the validation set. What is the performance of your
classifier for each of the languages Finnish, Japanese and Russian?



####  1. Exploring the dataset

In [ ]:
from huggingface_hub import notebook_login

notebook_login() # using token

In [ ]:
import pandas as pd

splits = {'train': 'train.parquet', 'validation': 'validation.parquet'}
df = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["train"])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
print(df.head)

<bound method NDFrame.head of                                                 question  \
0      উইকিলিকস কত সালে সর্বপ্রথম ইন্টারনেটে প্রথম তথ...   
1               দ্বিতীয় বিশ্বযুদ্ধে কোন দেশ পরাজিত হয় ?   
2      মার্কিন যুক্তরাষ্ট্রের সংবিধান অনুযায়ী মার্কিন...   
3      আরব-ইসরায়েলি যুদ্ধে আরবের মোট কয়জন সৈন্যের মৃ...   
4              বিশ্বে প্রথম পুঁজিবাদী সমাজ কবে গড়ে ওঠে ?   
...                                                  ...   
15321              కోళ్లు ఎక్కువగా ఏ దేశంలో కనిపిస్తాయి?   
15322       క్షయ వ్యాధికి విరుగుడు ఏ దేశంలో కనుగొన్నారు?   
15323                 ఖురాన్ ఏ అరబ్బీ భాషలో ఎవరు రాసారు?   
15324  టెక్సస్ రాష్ట్రంలోని అతిపెద్ద మానవ నిర్మితం ఏది ?   
15325         తమిళనాడులో రాష్ట్ర మొదటి ముఖ్యమంత్రి ఎవరు?   

                                                 context lang  answerable  \
0      WikiLeaks () is an international non-profit or...   bn        True   
1      The war in Europe concluded with an invasion o...   bn        True   
2      Same-sex ma

In [ ]:
df.describe()

,answer_start
count,15326.000000
mean,157.062769
std,226.748482
min,-1.000000
25%,13.000000
50%,78.000000
75%,210.000000
max,3964.000000


In [ ]:
# Train and validation sets
train_df = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["train"])
val_df = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["validation"])

# Explore the dataset columns (training and validation)
print("Training Data Columns:", train_df.columns)
print("Validation Data Columns:", val_df.columns)

Training Data Columns: Index(['question', 'context', 'lang', 'answerable', 'answer_start', 'answer',
       'answer_inlang'],
      dtype='object')
Validation Data Columns: Index(['question', 'context', 'lang', 'answerable', 'answer_start', 'answer',
       'answer_inlang'],
      dtype='object')


In [ ]:
finnish_questions = df[df['lang'] == 'fi']
japanese_questions = df[df['lang'] == 'ja']
russian_questions = df[df['lang'] == 'ru']

# Summarize basic statistics
print("Statistics for Finnish Questions:")
print(finnish_questions.describe(include='all'))  # or include only numeric features by default
print("\nStatistics for Japanese Questions:")
print(japanese_questions.describe(include='all'))
print("\nStatistics for Russian Questions:")
print(russian_questions.describe(include='all'))

Statistics for Finnish Questions:
                                                question  \
count                                               2126   
unique                                              2100   
top     Kuinka monta peliä Final Fantasy-sarjaan kuuluu?   
freq                                                   3   
mean                                                 NaN   
std                                                  NaN   
min                                                  NaN   
25%                                                  NaN   
50%                                                  NaN   
75%                                                  NaN   
max                                                  NaN   

                                                  context  lang answerable  \
count                                                2126  2126       2126   
unique                                               2054     1          2   
top     Wes

We can see, that there are 2301 Japanese questions (2100 unique), 2126 Finnish questions (1743 unique) and 1983 Russian questions (1612 unique).

#### 2.  5 most common words

In [ ]:
!pip install googletrans==4.0.0-rc1

In [ ]:
from collections import Counter
from googletrans import Translator
translator = Translator()

In [ ]:
# Find 5 most common words in Finnish in df dataset

finnish_questions = df[df['lang'] == 'fi']['question']

# Combine all questions into a single string
all_questions_text = ' '.join(finnish_questions.astype(str))

# Tokenize the text
words = all_questions_text.lower().split()

# Count the frequency of each word
word_counts = Counter(words)

# Get the 5 most common words
top_5_words = word_counts.most_common(5)

# Translate words dynamically
translated_top_5 = []
for word, count in top_5_words:
    translation = translator.translate(word, src='fi', dest='en').text  # Translate word from Finnish to English
    translated_top_5.append((word, count, translation))

# Print the top 5 words with translations
print("Top 5 most common words in Finnish questions with English translations:")
for word, count, translation in translated_top_5:
    print(f"{word} (count: {count}) - English: {translation}")

# print("Top 5 most common words in Finnish questions:", top_5_words)


Top 5 most common words in Finnish questions with English translations:
on (count: 582) - English: there is
mikä (count: 328) - English: What
milloin (count: 287) - English: When
vuonna (count: 227) - English: in
kuka (count: 215) - English: Who


In [ ]:
# Find 5 most common words in Russian in df dataset
russian_questions = df[df['lang'] == 'ru']['question']

all_questions_text = ' '.join(russian_questions.astype(str))

words = all_questions_text.lower().split()

word_counts = Counter(words)

top_5_words = word_counts.most_common(5)

# Translate words dynamically
translated_top_5 = []
for word, count in top_5_words:
    translation = translator.translate(word, src='ru', dest='en').text  # Translate word from Russian to English
    translated_top_5.append((word, count, translation))

# Print the top 5 words with translations
print("Top 5 most common words in Russian questions with English translations:")
for word, count, translation in translated_top_5:
    print(f"{word} (count: {count}) - English: {translation}")

#print("Top 5 most common words in Russian questions:", top_5_words)




Top 5 most common words in Russian questions with English translations:
в (count: 978) - English: V
сколько (count: 426) - English: How many
на (count: 385) - English: on
когда (count: 251) - English: When
кто (count: 209) - English: Who


For both Finnish and Russian it appears the the most common words are question
words or "connecting"/grammar words.

Let's proceed to Japanese:

In [ ]:
# Find 5 most common words in Japanese in df dataset
japanese_questions = df[df['lang'] == 'ja']['question']

all_questions_text = ' '.join(japanese_questions.astype(str))

words = all_questions_text.lower().split()

word_counts = Counter(words)

top_5_words = word_counts.most_common(5)

print("Top 5 most common words in Japanese questions:", top_5_words)

Top 5 most common words in Japanese questions: [('pygが最初に発表した曲は何', 3), ('オーロラ号の漂流時、オーロラ号に乗組員は何人いた？', 3), ('カンザスシティが設立されたのはいつ', 3), ('クロイツ群が初めて発見されたのはいつ', 3), ('ライド', 2)]


We can see that instead of actually counting words in japanese, it counted whole sentences. We'll try to use a specific tokenizer for japanese:

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer_ja = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ja-en")
model_ja = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ja-en")

#tokenizer_fi = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-fi-en")
#model_fi = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-fi-en")

#tokenizer_ru = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ru-en")
#model_ru = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ru-en")



/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
def tokenize_japanese_sentences(sentence):
    tokens = tokenizer_ja.tokenize(sentence)
    return " ".join(tokens)

# Apply tokenization to the 'japanese_questions' series
tokenized_japanese_questions = japanese_questions.apply(tokenize_japanese_sentences)

# Print the original and tokenized questions
for original, tokenized in zip(japanese_questions.head(5), tokenized_japanese_questions.head(5)):
    print(f"Original: {original}")
    print(f"Tokenized: {tokenized}")
    print("-" * 50)

Original: ポーランドで農地改革が行われたことがある？
Tokenized: ▁ポーランド で 農 地 改革 が 行わ れた こと がある ?
--------------------------------------------------
Original: ビスカヤ県で初めて進出した大規模鉱業会社は何？
Tokenized: ▁ビ スカ ヤ 県 で 初めて 進 出 した 大 規模 鉱 業 会社 は 何 ?
--------------------------------------------------
Original: 古代ローマ帝国はいつ起きた？
Tokenized: ▁古代 ローマ 帝国 は いつ 起き た ?
--------------------------------------------------
Original: スペイン・バスク州の州都はどこですか？
Tokenized: ▁スペイン ・ バス ク 州 の 州 都 は どこ です か ?
--------------------------------------------------
Original: イタリア王国海軍は第一次世界大戦中に何隻戦艦をつくった？
Tokenized: ▁イタリア 王国 海軍 は 第一 次 世界 大戦 中 に 何 隻 戦艦 を つく った ?
--------------------------------------------------


Now that the Japanese questions are properly tokenized, we can proceed to find the most common words:

In [ ]:
all_questions_text = ' '.join(tokenized_japanese_questions.astype(str))

words = all_questions_text.lower().split()

word_counts = Counter(words)

top_5_words = word_counts.most_common(5)

# Translate words dynamically
translated_top_5 = []
for word, count in top_5_words:
    translation = translator.translate(word, src='ja', dest='en').text  # Translate word from Japanese to English
    translated_top_5.append((word, count, translation))

# Print the top 5 words with translations
print("Top 5 most common words in Japanese questions with English translations:")
for word, count, translation in translated_top_5:
    print(f"{word} (count: {count}) - English: {translation}")

Top 5 most common words in Japanese questions with English translations:
は (count: 2213) - English: teeth
? (count: 1644) - English: ?
の (count: 1622) - English: of
・ (count: 646) - English: ・
何 (count: 542) - English: what


Here, because we pre-tokenized the data, punctuation gets treated like words. For that reason the question mark and period got into the 5 most commonly used "words".

Another thing to note is the "は" token. Altough it can be translated to "teeth", it is most commonly used as grammatical particle - used to indicate the subject of the sentence. It is a frequently used particle which explains why we see it so much.

Similarly, the "の" token is akin to the possessive 's in english, which can explain its high frequency in the data.

We can try to remove the grammar particles and punctuation tokens in order to find the most common "words". It is important to note the defintion of what a "word" is can be different depending on the language (and the person you're asking).

In [ ]:
import re

# Function to remove punctuation and unwanted particles
def clean_text(text):
    # Remove punctuation using regex
    text = re.sub(r'[^\w\s]', '', text)  # Removes non-word characters (punctuation)

    # List of common Japanese particles to filter out
    particles = {'は', 'が', 'を', 'に', 'の', 'で', 'と', 'や', 'も', 'から', 'まで', 'へ', 'か'}

    # Split the text and filter out particles
    words = text.lower().split()
    cleaned_words = [word for word in words if word not in particles]

    return cleaned_words

# Convert tokenized questions into a single string
all_questions_text = ' '.join(tokenized_japanese_questions.astype(str))

# Clean the text by removing punctuation and filtering out particles
cleaned_words = clean_text(all_questions_text)

# Count word frequencies
word_counts = Counter(cleaned_words)

# Get the top 5 most common words
top_5_words = word_counts.most_common(5)

# Translate words dynamically
translated_top_5 = []
for word, count in top_5_words:
    translation = translator.translate(word, src='ja', dest='en').text  # Translate word from Japanese to English
    translated_top_5.append((word, count, translation))

# Print the top 5 words with translations
print("Top 5 most common words in Japanese questions with English translations:")
for word, count, translation in translated_top_5:
    print(f"{word} (count: {count}) - English: {translation}")

Top 5 most common words in Japanese questions with English translations:
何 (count: 542) - English: what
した (count: 469) - English: did
いつ (count: 459) - English: when
誰 (count: 315) - English: who
です (count: 216) - English: is


As we can see, the filtered Japanese results are similar to the Finnish and Russian results.


####  3. Rule-based Classifier

Classifier that predicts whether a question is answerable or impossible, only using the document (context) and question. You may use machine translation as a component.

In [ ]:
def predict_answerability(context, question, lang):
    translated_question = translator.translate(question, src=lang, dest='en').text

    # Check if the translated question is in the context
    keywords = translated_question.lower().split()

    # Search for a keywords
    for keyword in keywords:
        if keyword in context.lower():
            return 'answerable',translated_question

    # If no keywords are found, the question is impossible
    return 'impossible', translated_question


# Test the classifier on the first 5 questions
for index, row in val_df[val_df['lang'] == 'fi'].head(10).iterrows():
    context = row['context']
    question = row['question']
    lang = row['lang']
    prediction, translated_question = predict_answerability(context, question, lang)
    print(f"Question: {question}")
    print(f"Translated question: {translated_question}")
    print(f"Context: {context}")
    print(f"Predicted: {prediction}")
    print(f"Actual: {row['answerable']}")
    print("-" * 50)


Question: Missä maassa Jack Churchill syntyi?
Translated question: In which country was Jack Churchill born?
Context: Churchill was born at Colombo, British Ceylon to Alec Fleming "Alex" Churchill (1876–1961), later of Hove, East Sussex and Elinor Elizabeth, daughter of John Alexander Bond Bell, of Kelnahard, County Cavan, Ireland, and of Dimbula, Ceylon. Alec, of a family long settled at Deddington, Oxfordshire, had been District Engineer in the Ceylon Civil Service, in which his father, John Fleming Churchill (1829–1894), had also served. Soon after Jack's birth, the family returned to Dormansland, Surrey, where his younger brother, Major-General Thomas Bell Lindsay Churchill (1907–1990), C.B., C.B.E., M.C. was born. In 1910, the Churchills moved to Hong Kong when Alec Fleming Churchill was appointed as Director of Public Works; he also served as a member of the Executive Council. The Churchills' third and youngest son, Robert Alec Farquhar Churchill—later a Lieutenant in the Royal N

In [ ]:
#!git clone https://github.com/sofi388/nlp_ucph.git

Cloning into 'nlp_ucph'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.


In [ ]:
#%cd nlp_ucph

/content/nlp_ucph


In [ ]:
#%cd /content/nlp_ucph

/content/nlp_ucph


In [1]:
#!find /content -name "NLP_Project.ipynb"